In [1]:
import pandas as pd

In [2]:
# read in csv
path = "PaydayVendorLocations.csv"

df = pd.read_csv(path)

df.head()

,Zip Code,Payday Vendor,Address,Lat,Lng
0,90001,ACE Cash Express,"1232 E Florence Ave, Los Angeles, CA 90001, Un...",33.974583,-118.252963
1,90001,Easy Financial,"6003 Compton Ave, Los Angeles, CA 90001, Unite...",33.985552,-118.248060
2,90002,ACE Cash Express,"10005 S Central Ave, Los Angeles, CA 90002, Un...",33.945482,-118.254526
3,90002,The Check Cashing Place,"2284 E Firestone Blvd, Los Angeles, CA 90002, ...",33.957857,-118.231333
4,90002,Continental Currency Services,"1669 E 103rd St, Los Angeles, CA 90002, United...",33.943856,-118.244298


In [12]:
# select my half of dataset
my_df = df[df["Zip Code"] > 93311].reset_index(drop = True)
print(len(my_df))
my_df.head()

1577


,Zip Code,Payday Vendor,Address,Lat,Lng
0,93312,OneMain Financial,"9680 Hageman Rd ste d, Bakersfield, CA 93312, ...",35.400366,-119.109465
1,93401,Payday Loans San Luis Obispo,"San Luis Obispo, CA 93401, United States",35.282752,-120.659616
2,93401,"check n go, Check `n Go Store Directory","3970 Broad St, San Luis Obispo, CA 93401, Unit...",35.249283,-120.643179
3,93401,OneMain Financial,"3920 Broad St Ste 5, San Luis Obispo, CA 93401...",35.249566,-120.641624
4,93401,Mid Coast Capital,"688 California Blvd, San Luis Obispo, CA 93401...",35.288666,-120.656548


In [11]:
# drop banks and credit unions

# list vendors
vendors = my_df["Payday Vendor"].unique().tolist()

# get list of vendors with "bank" or "credit union" in name
bank_or_CU = []
for v in vendors:
    if ("bank" in v.lower()) or ("credit union" in v.lower()):
        bank_or_CU.append(v)


len(bank_or_CU)

112

In [13]:
# get indices of all rows with bank_or_CU vendor
bank_drops = []
for row in my_df.iterrows():
    if row[1]["Payday Vendor"] in bank_or_CU:
        bank_drops.append(row[0])

# drop those rows
nobank_df = my_df.drop(bank_drops)
len(nobank_df)

1120

In [14]:
# get identical addresses into list
addresses = nobank_df["Address"].value_counts()

non_unique_addresses = addresses[addresses > 1]

non_uniques = pd.DataFrame(non_unique_addresses).reset_index()
non_uniques = non_uniques["index"].tolist()

len(non_uniques)

13

In [15]:
# decide which address to drop
address_drops = []

for n in non_uniques:
    locs = nobank_df[my_df["Address"] == n]
    print(locs["Payday Vendor"])
    keep1 = input("keep 1? y/n ")
    keep2 = input("keep 2? y/n ")
    
    if keep1 == "n":
        address_drops.append(locs.index.tolist()[0])
        
    if keep2 == "n":
        address_drops.append(locs.index.tolist()[1])
        
    

<ipython-input-15-a8aeb77792cb>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  locs = nobank_df[my_df["Address"] == n]


1303    California Check Cashing Stores
1314                      Western Union
Name: Payday Vendor, dtype: object
keep 1? y/n y
keep 2? y/n n
262    California Budget Finance
264             ACE Cash Express
Name: Payday Vendor, dtype: object
keep 1? y/n n
keep 2? y/n y
973                                Check `n Go
976    check n go, Check `n Go Store Directory
Name: Payday Vendor, dtype: object
keep 1? y/n y
keep 2? y/n n
488                   Cash 1
492    Buckeye Check Cashing
Name: Payday Vendor, dtype: object
keep 1? y/n n
keep 2? y/n y
1074            Check Max Plus
1077    Frontera Cash And Loan
Name: Payday Vendor, dtype: object
keep 1? y/n y
keep 2? y/n n
621                                Check `n Go
625    check n go, Check `n Go Store Directory
Name: Payday Vendor, dtype: object
keep 1? y/n y
keep 2? y/n n
1494                                Check `n Go
1498    check n go, Check `n Go Store Directory
Name: Payday Vendor, dtype: object
keep 1? y/n y
keep 2? y/n n
125      

In [23]:
# drop addresses
cleaner_df = nobank_df.drop(address_drops)
len(cleaner_df)

1106

In [35]:
# new list of vendors
vendors_cleaner = cleaner_df["Payday Vendor"].value_counts().reset_index()["index"].tolist()
vendors_cleaner

['Advance America',
 'Check Into Cash',
 'California Check Cashing Stores',
 'Oportun',
 'OneMain Financial',
 'ACE Cash Express',
 'check n go, Check `n Go Store Directory',
 'Check `n Go',
 'Money Mart',
 'Montana Capital Car Title Loans',
 'Cash 1',
 'Western Union',
 'Lendmark Financial Services LLC',
 'Raymond James Financial Services',
 'DolEx Dollar Express',
 '5 Star Car Title Loans',
 'Family Car Title Loans',
 'Check Center',
 'Thrivent Financial for Lutherans',
 'Car Title Loans California',
 'Fast Auto and Payday Loans',
 'Allied Cash Advance',
 'TitleMax Title Loans',
 'MoneyGram',
 'American AgCredit',
 'CashMax',
 'USA Cash Services',
 'Thrivent Financial',
 'Best Collateral',
 'Fairway Independent Mortgage Corporation',
 'EZ Cash Plus INC.',
 'Thrivent Financial-Lutherans',
 'Fast Money Car Title Loans',
 'United Indian Health Services',
 'Payday Services Plus',
 'Express Cash Advance',
 'MVP Car Title Loans',
 'Lucky Check Cashing',
 'Speedy Cash',
 'Consumer Credit Co

In [25]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\isabe\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [36]:
def checkstring(string, var1, var2):
    check1 = string.lower() in var1.lower()
    check2 = string.lower() in var2.lower()
    return check1 and check2

In [64]:
cleaning_dict = {}
vals = []

for i in range(len(vendors_cleaner)):
    for j in range(i + 1, len(vendors_cleaner)):
        n1 = vendors_cleaner[i]
        n2 = vendors_cleaner[j]
        
        titleloan = not checkstring("title loan", n1, n2)
        paydayloan = not checkstring("payday loan", n1, n2)
        cashplus = not checkstring("cash plus", n1, n2)
        
        if titleloan and paydayloan:
            rat = fuzz.partial_ratio(n1, n2)
            if rat > 90:
                if n1 in vals:
                    cleaning_dict[n2] = n1
                    print("hi")
                elif n2 in vals:
                    cleaning_dict[n1] = n2
                    print("bye")
                    
                else:
                    keep = input(f"{n2} | {n1} y/n/r? ") # yes = add to dict, r = reverse order and add
                    if keep == "y":
                        cleaning_dict[n1] = n2
                        vals.append(n2)
                    elif keep == "r":
                        cleaning_dict[n2] = n1
                        vals.append(n1)

California Check Cashing | California Check Cashing Stores y/n/r? y
Oportun Headquarters | Oportun y/n/r? y
Check `n Go | check n go, Check `n Go Store Directory y/n/r? r
Dolex Dollar Express | DolEx Dollar Express y/n/r? y
Thrivent Financial | Thrivent Financial for Lutherans y/n/r? r
Thrivent Financial-Lutherans | Thrivent Financial y/n/r? y
Nathan Mahnke ChFC® , FIC , RICP® - Thrivent Financial | Thrivent Financial y/n/r? y


KeyboardInterrupt: Interrupted by user

In [53]:
for i in range(len(cleaning_dict)):
    cleaning_dict[i] = cleaning_dict[i] + ","


["'California Check Cashing':'California Check Cashing Stores',", "'Oportun Headquarters':'Oportun',", "'check n go, Check `n Go Store Directory':'Check `n Go',", "'Dolex Dollar Express':'DolEx Dollar Express',", "'Thrivent Financial for Lutherans':'Thrivent Financial',", "'Thrivent Financial-Lutherans':'Thrivent Financial',", "'Nathan Mahnke ChFC® , FIC , RICP® - Thrivent Financial':'Thrivent Financial',", "'Thrivent Financial Templeton':'Thrivent Financial',", "'Thrivent Financial - Lutherans':'Thrivent Financial',", "'Fairway Independent Mortgage':'Fairway Independent Mortgage Corporation',", "'Cassandra Askew Leisure | Fairway Independent Mortgage Corporation Loan Officer':'Fairway Independent Mortgage Corporation',", "'EZ Cash Plus':'EZ Cash Plus INC.',", "'United Indian Health Services - Potawot Health Village':'United Indian Health Services',", "'RPM Lenders Title Loans':'RPM Lenders',", "'Cash Now ETC.':'Cash Now',", "'Cash Now Fremont':'Cash Now',", "'Norco Texaco':'Texaco',",

In [60]:
fixed_dict = "\n".join(cleaning_dict)

In [62]:
clean_df = cleaner_df.replace({fixed_dict})

clean_df.head()

,Zip Code,Payday Vendor,Address,Lat,Lng
0,93312,OneMain Financial,"9680 Hageman Rd ste d, Bakersfield, CA 93312, ...",35.400366,-119.109465
1,93401,Payday Loans San Luis Obispo,"San Luis Obispo, CA 93401, United States",35.282752,-120.659616
2,93401,"check n go, Check `n Go Store Directory","3970 Broad St, San Luis Obispo, CA 93401, Unit...",35.249283,-120.643179
3,93401,OneMain Financial,"3920 Broad St Ste 5, San Luis Obispo, CA 93401...",35.249566,-120.641624
4,93401,Mid Coast Capital,"688 California Blvd, San Luis Obispo, CA 93401...",35.288666,-120.656548


In [154]:
# # Combining similar offenses together
# no_null_crime_df = no_null_crime_df.replace(
#     {"Commercial Sex Acts": "Prostitution", "Assisting or Promoting Prostitution": "Prostitution"})
# no_null_crime_df

unbanked_df = unbanked_df.replace(
    {"California Check Cashing Stores":"California Check Cashing",
    "Oportun Headquarters": "Oportun",
    "check n go, Check `n Go Store Directory": "Check `n Go",
    "Dolex Dollar Express":"DolEx Dollar Express",
    'Thrivent Financial-Lutherans':"Thrivent Financial",
     'Thrivent Financial - Lutherans':"Thrivent Financial",
     'Nathan Mahnke ChFC® , FIC , RICP® - Thrivent Financial':"Thrivent Financial",
     'Thrivent Financial Templeton':"Thrivent Financial",
    "EZ Cash Plus":"EZ Cash Plus INC.",
    'Fairway Independent Mortgage':'Fairway Independent Mortgage Corporation',
     'Cassandra Askew Leisure | Fairway Independent Mortgage Corporation Loan Officer':'Fairway Independent Mortgage Corporation',
    'United Indian Health Services - Potawot Health Village':'United Indian Health Services',
    'Alta One Federal Credit Union':'AltaOne Federal Credit Union',
    'RPM Lenders Title Loans':'RPM Lenders',
    'MVP Car Title Loans Concord':'MVP Car Title Loans',
'MVP Car Title Loans San Rafael':'MVP Car Title Loans',
#'Title Loans':'MVP Car Title Loans',
#'Cash Plus':'EZ Cash Plus',
'Cash Now Fremont':'Cash Now',
'Cash Now ETC.':'Cash Now',
'Navy Federal Credit Union - Restricted Access': "Federal Credit Union",
'Sonoma Federal Credit Union':'Federal Credit Union',
'Navy Federal Credit Union':'Federal Credit Union',
'Menlo Survey Federal Credit Union':'Federal Credit Union',
'OE Federal Credit Union':'Federal Credit Union',
'Mission City Federal Credit Union':'Federal Credit Union',
'Santa Clara County Federal Credit Union':'Federal Credit Union',
'Central Coast Federal Credit Union':'Federal Credit Union',
'Caliber Home Loans':'Caliber Home Loans - Adobe Mortgage Group',
'Cassandra Askew Leisure | Fairway Independent Mortgage Corporation Loan Officer':'Fairway Independent Mortgage',
#'OE Federal Credit Union':'SRI Federal Credit Union',
#'Title Loans':'MVP Car Title Loans Concord',
'Norco Texaco':'Texaco',
'Stearns Lending':'Stearns Lending Inc',
'800 Loan Mart':'1800 Loan Mart',
#'Title Loans':'Paid Today Car Title Loans',
#'Title Loans':'Lead Car Title Loans Woodland',
#'Payday Loans':'Express Payday Loans',
'Lisa Luperini- Caliber Home Loans Inc.':'Caliber Home Loans',
'Caliber Home Loans - The Superior Team':'Caliber Home Loans',
'Don Krause - Caliber Home Loans':'Caliber Home Loans',
#'Title Loans':'Get Auto Title Loans Fresno CA',
'Bob\'s Humboldt Pawndemonium':'Humboldt Pawn',
'Payday Loans San Luis Obispo':'Payday Loans',
'Brian Humphrey - Supreme Lending':'Supreme Lending',
'Mike Ervin at Supreme Lending':'Supreme Lending',
#'OE Federal Credit Union':'Menlo Survey Federal Credit Union',
#'Title Loans':'Swift Title Loans',
'Swift Title Loans South Lake Tahoe':'Swift Title Loans',
'A-1 Check Cashing':'A1 Check Cashing',
'A1 Check Cashing of Corning':'A1 Check Cashing',
'American Pacific Mortgage':'American Pacific Mortgage Tehachapi',
'A1 Check Cashing of Corning':'A1 Check Cashing',
'Chris Lamm - Commerce Home Mortgage':'Commerce Home Mortgage',
'Community Check Cashing East 12th Street, Oakland, CA':'Community Check Cashing'})

new_vends = unbanked_df["Payday Vendor"].value_counts()
new_vends_top = new_vends[new_vends > 5]
new_vends_top.sum()

756

In [152]:
unbanked_df.tail()

,index,Zip Code,Payday Vendor,Address,Lat,Lng
1565,3152,96150,Sierra Central Credit Union,"3668 Lake Tahoe Blvd, South Lake Tahoe, CA 961...",38.948457,-119.956407
1567,3154,96150,MoneyGram,"3471 Lake Tahoe Blvd, South Lake Tahoe, CA 961...",38.946675,-119.964723
1573,3160,96161,Sierra Mountain Mortgage Inc,"10098 Jibboom St # 201, Truckee, CA 96161, Uni...",39.328434,-120.186173
1575,3162,96161,"Happy Investments, Inc. Truckee Ca","11260 Donner Pass Road C1 # 1014-G, Truckee, C...",39.327165,-120.205797
1576,3163,96161,Wells Fargo Insurance Services,"Truckee, CA 96161, United States",39.327962,-120.183253


In [157]:
addresses_fuzzy = unbanked_df["Address"].unique().tolist()
# vends = counts2["index"].tolist()




In [162]:
fuzz_98 = []
fuzz_97 = []
fuzz_96 = []
fuzz_95 = []

for i in range(len(addresses_fuzzy)):
    for j in range(i + 1, len(addresses_fuzzy)):
        rat = fuzz.ratio(addresses_fuzzy[i], addresses_fuzzy[j])
        if rat == 98:
            print(f"'{addresses_fuzzy[j]}' | '{addresses_fuzzy[i]}'")
            fuzz_98.append((addresses_fuzzy[j], addresses_fuzzy[i]))
        elif rat == 97:
            fuzz_97.append((addresses_fuzzy[j], addresses_fuzzy[i]))
        elif rat == 96:
            fuzz_96.append((addresses_fuzzy[j], addresses_fuzzy[i]))
        elif rat == 95:
            fuzz_95.append((addresses_fuzzy[j], addresses_fuzzy[i]))

'501 Old Mammoth Rd, Mammoth Lakes, CA 93546, United States' | '101 Old Mammoth Rd, Mammoth Lakes, CA 93546, United States'
'841 Tucker Rd, Tehachapi, CA 93561, United States' | '821 Tucker Rd, Tehachapi, CA 93561, United States'
'2739 Whitson St, Selma, CA 93662, United States' | '2734 Whitson St, Selma, CA 93662, United States'
'2325 E McKinley Ave, Fresno, CA 93703, United States' | '2425 E McKinley Ave, Fresno, CA 93703, United States'
'4959 E Kings Canyon Rd, Fresno, CA 93727, United States' | '4939 E Kings Canyon Rd, Fresno, CA 93727, United States'
'1577 N Sanborn Rd, Salinas, CA 93905, United States' | '1571 N Sanborn Rd, Salinas, CA 93905, United States'
'628 Williams Rd, Salinas, CA 93905, United States' | '624 Williams Rd, Salinas, CA 93905, United States'
'510 Veterans Blvd, Redwood City, CA 94063, United States' | '590 Veterans Blvd, Redwood City, CA 94063, United States'
'3011 Mission St, San Francisco, CA 94110, United States' | '3001 Mission St, San Francisco, CA 94110,

In [174]:
possibles = []
for x in fuzz_98:
#     place1 = unbanked_df[unbanked_df["Address"] == x[0]]["Payday Vendor"].values[0]
#     place2 = unbanked_df[unbanked_df["Address"] == x[1]]["Payday Vendor"].values[0]
#     print(f"{place1} | {place2}")
#     print(fuzz.ratio(place1, place2))
    if x[0][0:4] == x[1][0:4]:
        print(x)
        q = input("y/n? ")
        if q == "y":
            possibles.append(x)

('2414 W Kettleman Ln Suite 202, Lodi, CA 95242, United States', '2414 W Kettleman Ln Suite 200, Lodi, CA 95242, United States')
y/n? n
('14520 Mono Way Suite 130, Sonora, CA 95370, United States', '14520 Mono Way Suite 110, Sonora, CA 95370, United States')
y/n? n
('635 E Hwy 20 C, Upper Lake, CA 95485, United States', '635 E Hwy 20 E, Upper Lake, CA 95485, United States')
y/n? n
('635 E Hwy 20 F, Upper Lake, CA 95485, United States', '635 E Hwy 20 E, Upper Lake, CA 95485, United States')
y/n? n
('635 E Hwy 20 F, Upper Lake, CA 95485, United States', '635 E Hwy 20 C, Upper Lake, CA 95485, United States')
y/n? n
('9250 Laguna Springs Dr #101, Elk Grove, CA 95758, United States', '9250 Laguna Springs Dr #210, Elk Grove, CA 95758, United States')
y/n? n
('1510 W El Camino Ave, Sacramento, CA 95833, United States', '1510-C W El Camino Ave, Sacramento, CA 95833, United States')
y/n? y
('151 W McKnight Way b, Grass Valley, CA 95949, United States', '151 W McKnight Way, Grass Valley, CA 9594

In [182]:
# merge localized title loan places
vends2 = unbanked_df["Payday Vendor"].unique().tolist()
print(len(vends2))

for i in range(len(vends2)):
    for j in range(i + 1, len(vends2)):
        rat = fuzz.partial_ratio(vends2[i], vends2[j])
        if rat > 90:
            print(f"'{vends2[j]}':'{vends2[i]}',")


418
'Payday Loan Today':'Payday Loans',
'Fast Auto and Payday Loans':'Payday Loans',
'Neighbor Payday Loans Salinas':'Payday Loans',
'LendUp - Alternative to Payday Loans':'Payday Loans',
'Get Payday Loans':'Payday Loans',
'Sacramento Payday Loans':'Payday Loans',
'Express Payday Loans':'Payday Loans',
'Title Loans':'Family Car Title Loans',
'Title Loans':'Montana Capital Car Title Loans',
'Federal Credit Union':'SESLOC Federal Credit Union',
'Oro Jewelry & Loan':'Atascadero Jewelry & Loan',
'Payday Advance':'Aloha Blue Payday Advance',
'Thrivent Financial for Lutherans':'Thrivent Financial',
'Title Loans':'Paid Today Car Title Loans',
'AltaOne Federal Credit Union':'Federal Credit Union',
'SRI Federal Credit Union':'Federal Credit Union',
'Delta Schools Federal Credit Union':'Federal Credit Union',
'Bay Federal Credit Union':'Federal Credit Union',
'Yolo Federal Credit Union':'Federal Credit Union',
'Self-Help Federal Credit Union':'Federal Credit Union',
'Title Loans':'TitleMax Title

In [199]:
CUs = []
not_CUs = []
for v in vends2:
    if "credit union" in v.lower():
        CUs.append(v)
    else:
        not_CUs.append(v)

len(CUs)


36

In [202]:
indices = []
for row in unbanked_df.iterrows():
    if row[1]["Payday Vendor"] in CUs:
        indices.append(row[0])

# test = next(unbanked_df.iterrows())
# # .apply() or .iterrows()
# test[1]["Zip Code"]

In [204]:
uncredited_df = unbanked_df.drop(indices)
uncredited_df.head()

,index,Zip Code,Payday Vendor,Address,Lat,Lng
0,1587,93312,OneMain Financial,"9680 Hageman Rd ste d, Bakersfield, CA 93312, ...",35.400366,-119.109465
1,1588,93401,Payday Loans,"San Luis Obispo, CA 93401, United States",35.282752,-120.659616
2,1589,93401,Check `n Go,"3970 Broad St, San Luis Obispo, CA 93401, Unit...",35.249283,-120.643179
3,1590,93401,OneMain Financial,"3920 Broad St Ste 5, San Luis Obispo, CA 93401...",35.249566,-120.641624
4,1591,93401,Mid Coast Capital,"688 California Blvd, San Luis Obispo, CA 93401...",35.288666,-120.656548


In [205]:
len(uncredited_df)

1100

In [206]:
uncredited_df.to_csv("IS_clean_v1.csv")

In [173]:

summarized_df = unbanked_df.replace({
    'Payday Loan Today':'Payday Loans',
    'Fast Auto and Payday Loans':'Payday Loans',
    'Neighbor Payday Loans Salinas':'Payday Loans',
    'LendUp - Alternative to Payday Loans':'Payday Loans',
    'Get Payday Loans':'Payday Loans',
    'Sacramento Payday Loans':'Payday Loans',
    'Express Payday Loans':'Payday Loans',
    
#     'Title Loans':'Family Car Title Loans',
#     'Title Loans':'Montana Capital Car Title Loans',
    'Federal Credit Union':'SESLOC Federal Credit Union',
    'Oro Jewelry & Loan':'Atascadero Jewelry & Loan',
    'Payday Advance':'Aloha Blue Payday Advance',
    'Thrivent Financial for Lutherans':'Thrivent Financial',
    'Title Loans':'Paid Today Car Title Loans',
    'AltaOne Federal Credit Union':'Federal Credit Union',
    'SRI Federal Credit Union':'Federal Credit Union',
    'Delta Schools Federal Credit Union':'Federal Credit Union',
    'Bay Federal Credit Union':'Federal Credit Union',
    'Yolo Federal Credit Union':'Federal Credit Union',
    'Self-Help Federal Credit Union':'Federal Credit Union',
    'Title Loans':'TitleMax Title Loans',
    'Title Loans':'5 Star Car Title Loans',
    'Title Loans':'Get Auto Title Loans Fresno CA',
    'EZ Cash Plus INC.':'Cash Plus',
    'Consumer Credit Counseling Services':'Credit Consulting Services',
    'My Credit Union':'Monterey Credit Union',
    'Title Loans':'Car Title Loans California',
    'Title Loans':'MVP Car Title Loans',
    'North Bay Credit Union':'My Credit Union',
    'Golden Valley Credit Union':'My Credit Union',
    'Check 2 Cash':'South San Francisco Check Cash',
    'Check N Cash':'South San Francisco Check Cash',
    'Title Loans':'Car Title Loans San Francisco',
    'Title Loans':'Get Auto Title Loans San Francisco CA',
    'Title Loans':'Hi Palo Alto Auto Car Title Loans',
    'Title Loans':'Get Auto Car Title Loans Concord Ca',
    'SAFE Credit Union - Corporate Office':'SAFE Credit Union',
    'Yolo Federal Credit Union':'Delta Schools Federal Credit Union',
    'Title Loans':'Fremont Gatl Title Loans',
    'Title Loans':'Get Auto Car Title Loans Lafayette Ca',
    'Title Loans':'Speed Car Title Loans',
    'Title Loans':'Oakland Gatl Title Loans',
    'Title Loans':'Oakland Best Car Title Loans',
    'Title Loans':'Online Title Loans Oakland CA',
    'Esperanza Travel & Check Cash':'Check 2 Cash',
    'Check N Cash':'Check 2 Cash',
    'Check N Cash':'Esperanza Travel & Check Cash',
    'Title Loans':'Family Title Loans',
    'Title Loans':'Quick Cash Funding LLC | Car Title Loans',
    'Title Loans':'TFC Title Loans - San Jose',
    'Title Loans':'Get Auto Title Loans San Jose CA',
    'Title Loans':'Online Car Title Loans San Jose CA',
    'Title Loans':'San Jose Car Title Loans',
    'Get Auto Title Loans Stockton CA':'Title Loans',
    'Online Car Title Loans Stockton CA':'Title Loans',
    'Fast Money Car Title Loans':'Title Loans',
    'Get Auto Car Title Loans Modesto Ca':'Title Loans',
    'Lead Car Title Loans Santa Rosa':'Title Loans',
    'Lead Car Title Loans Roseville':'Title Loans',
    'Lead Car Title Loans Woodland':'Title Loans',
    'Swift Title Loans':'Title Loans',
    'Little Lake Lending':'Wake Lending'})
    

'hel'

In [22]:


addresses = df["Address"].value_counts()

non_uniques = addresses[addresses > 1]

# Start from row: 93312 and higher  OneMainFinacial 


16161 Ventura Blvd, Encino, CA 91436, United States                3
110 Main St, Brawley, CA 92227, United States                      2
2612 Saviers Rd, Oxnard, CA 93033, United States                   2
231 17th St, Santa Ana, CA 92706, United States                    2
2951A Harbor St, Pittsburg, CA 94565, United States                2
2095 Central Ave, McKinleyville, CA 95519, United States           2
1063 G St, Arcata, CA 95521, United States                         2
7204 Canoga Ave, Canoga Park, CA 91303, United States              2
785 E Shaw Ave, Fresno, CA 93710, United States                    2
1772 Piner Rd, Santa Rosa, CA 95403, United States                 2
710 Main St, El Centro, CA 92243, United States                    2
2031 Main St, Oakley, CA 94561, United States                      2
1950 N Texas St, Fairfield, CA 94533, United States                2
1311 W Warner Ave, Santa Ana, CA 92704, United States              2
1312 S Harbor Blvd, Fullerton, CA 

In [10]:
counts = pd.DataFrame(df["Payday Vendor"].value_counts(), )
counts


,Payday Vendor
Advance America,192
Check Into Cash,173
ACE Cash Express,154
Oportun,141
California Check Cashing Stores,104
...,...
Title Loan,1
"12M Loans, LLC",1
Christopher Austin - Mortgage Lender,1
Emergicash,1
